In [2]:
# !wget https://raw.githubusercontent.com/jla-gardner/graph-pes/refs/heads/main/scripts/build-lammps.sh
!bash build-lammps.sh --help

Found graph-pes pair style at /Users/john/projects/graph-pes/src/graph_pes/pair_style
Usage: build-lammps.sh [OPTIONS]

This script builds a LAMMPS executable with support for 'pair_style graph_pes'.

It performs the following tasks:
  1. Locates the graph-pes installation
  2. Creates a conda environment with necessary dependencies
  3. Clones the LAMMPS repository
  4. Patches LAMMPS with graph-pes source code
  5. Builds LAMMPS with graph-pes support

Options:
  --help           Display this help message and exit
  --cpu-only       Build LAMMPS for CPU only (default: GPU enabled)
  --force-rebuild  Force rebuilding of conda environment and LAMMPS

The final executable will be located at:
  ./graph_pes_lmp_cpu_only (if --cpu-only is used)
  ./graph_pes_lmp (default, with GPU support)
